In [14]:
# ==============================================
# ⭐ FINAL ENSEMBLE RECOMMENDER (WEIGHTED + MIXED MODELS)
# ==============================================

import pandas as pd
import numpy as np

# ===============================
# 🔧 MODEL WEIGHTS (EDIT HERE)
# ===============================
W_APRIORI = 0.25
W_COLLAB  = 0.25
W_CONTENT = 0.25
W_RFM     = 0.25


def ensemble_recommend(product, top_n=5):
    product_norm = str(product).lower().strip()

    # Storage for (product, model, score)
    rows = []

    # ------------------------------------------
    # 1️⃣ APRIORI RULES
    # ------------------------------------------
    if 'rules' in globals() and not rules.empty:
        try:
            related = rules[
                rules['antecedents'].apply(
                    lambda s: product_norm in [str(i).lower().strip() for i in s]
                )
            ]

            for _, r in related.iterrows():
                for c in r["consequents"]:
                    c = str(c).lower().strip()
                    lift_score = float(r.get("lift", 1))
                    rows.append((c, "Apriori", lift_score))
        except Exception:
            pass

    # ------------------------------------------
    # 2️⃣ COLLABORATIVE FILTERING
    # ------------------------------------------
    try:
        recs = collaborative_recommend(product_norm, top_n=10)
        for rank, item in enumerate(recs):
            item = str(item).lower().strip()
            score = 1 / (rank + 1)      # higher rank → higher weight
            rows.append((item, "Collaborative", score))
    except:
        pass

    # ------------------------------------------
    # 3️⃣ CONTENT-BASED TF-IDF
    # ------------------------------------------
    try:
        recs = content_recommend(product_norm, top_n=10)
        for rank, item in enumerate(recs):
            item = str(item).lower().strip()
            score = 1 / (rank + 1)
            rows.append((item, "Content", score))
    except:
        pass

    # ------------------------------------------
    # 4️⃣ RFM MODEL
    # ------------------------------------------
    try:
        recs = rfm_recommend(product_norm, top_n=10)
        for rank, item in enumerate(recs):
            item = str(item).lower().strip()
            score = 1 / (rank + 2)      # slightly weaker by design
            rows.append((item, "RFM", score))
    except:
        pass

    # ------------------------------------------
    # ❗ If no recs — exit
    # ------------------------------------------
    if not rows:
        return pd.DataFrame(columns=["Product", "Model", "Score"])

    df_all = pd.DataFrame(rows, columns=["Product", "Model", "raw_score"])

    # Normalize raw scores inside each model
    df_all["norm_score"] = df_all.groupby("Model")["raw_score"].transform(
        lambda s: s / s.max()
    )

    # ----------------------------------------------------
    # 🔢 Apply ensemble weights
    # ----------------------------------------------------
    weight_map = {
        "Apriori": W_APRIORI,
        "Collaborative": W_COLLAB,
        "Content": W_CONTENT,
        "RFM": W_RFM
    }

    df_all["weighted_score"] = df_all.apply(
        lambda x: x["norm_score"] * weight_map.get(x["Model"], 0), axis=1
    )

    # ----------------------------------------------------
    # 🧹 Deduplicate → keep highest-scoring source
    # ----------------------------------------------------
    final_df = (
        df_all.sort_values("weighted_score", ascending=False)
              .drop_duplicates(subset=["Product"])
              .head(top_n)
              .reset_index(drop=True)
    )

    # ----------------------------------------------------
    # Restore original casing if possible
    # ----------------------------------------------------
    if "df_content" in globals():
        mapping = {d.lower(): d for d in df_content["Description"].astype(str)}
        final_df["Product"] = final_df["Product"].map(mapping).fillna(final_df["Product"])

    return final_df[["Product", "Model", "weighted_score"]]


In [15]:
test_product = "set 5 red spotty lid glass bowls"
recommendations = ensemble_recommend(test_product, top_n=5)

print("\n🛍️ Recommended Products:")
print(recommendations)



🛍️ Recommended Products:
                               Product          Model  weighted_score
0   round container set of 5 retrospot  Collaborative        0.250000
1  set 5 red retrospot lid glass bowls        Content        0.250000
2   white hanging heart t light holder            RFM        0.250000
3             regency cakestand 3 tier            RFM        0.166667
4                      red spotty bowl        Content        0.125000
